In [21]:
import torch
from sparsembed import model
from transformers import AutoModelForMaskedLM, AutoTokenizer
import sparsembed
import pandas as pd
import numpy as np
import json

## Opzetten van de functies

#### 1. cosine_similarity(a, b): 
Deze functie berekent de cosine similarity tussen twee vectoren. Cosine similarity is een maatstaf die aangeeft hoe vergelijkbaar twee vectoren zijn, ongeacht hun grootte.

#### 2. embed(text_to_embed):
Deze functie zet een gegeven tekst om in sparse embeddings met behulp van het Splade-model. Embeddings zijn vectorrepresentaties van tekst die de betekenis van de tekst in een numerieke vorm vastleggen.

#### 3. create_vector_db(file_path, output_path='vector_db.json'):
Deze functie maakt een vector-database van een tekstbestand en slaat deze op in een JSON-bestand. Het leest de zinnen uit een bestand, genereert embeddings voor elke zin, en slaat deze embeddings samen met de originele zinnen op in een JSON-bestand.

#### 4. load_vector_data(file_path):
Deze functie laadt vectorgegevens uit een JSON-bestand en zet ze terug om naar tensors. Dit stelt ons in staat om de opgeslagen vectorrepresentaties opnieuw te gebruiken.

#### 5. get_splade_embeddings(example_question):
Deze functie zet een gegeven vraag om in sparse embeddings met behulp van het Splade-model. Dit is vergelijkbaar met de embed functie, maar specifiek voor het verwerken van vragen.

#### 6. find_most_similar_sentence(example_question, loaded_tensors, loaded_sentences):
Deze functie vindt de meest vergelijkbare zin ten opzichte van een gegeven vraag uit een lijst van vooraf geladen zinnen. Door cosine similarity te berekenen tussen de vector van de vraag en de vectoren van de geladen zinnen, kunnen we de meest vergelijkbare zin identificeren.

In [22]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> np.float64:

    """
    Computes the cosine similarity between two vectors.

    Args:
        a (np.ndarray): First input vector.
        b (np.ndarray): Second input vector.

    Returns:
        np.float64: Cosine similarity between vector a and vector b.
    """
    dot = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot / ( norm_a * norm_b )
    
cos_sim = np.vectorize(cosine_similarity)

In [23]:
def embed(text_to_embed: str) -> dict:
    
    """
    Encodes a given text into sparse embeddings using the Splade model.

    Args:
        text_to_embed (str): The text to be encoded into embeddings.

    Returns:
        dict: A dictionary containing the sparse activations from the embeddings.
    """
    
    splade_model = sparsembed.model.Splade(
    model=AutoModelForMaskedLM.from_pretrained("naver/splade_v2_max"),
    tokenizer=AutoTokenizer.from_pretrained("naver/splade_v2_max")
    )

# Encode text into embeddings using Splade
    with torch.no_grad():
        # Encode the input text into embeddings
        embeddings = splade_model.encode(
            texts=[text_to_embed],  # Provide the input text as a list
            truncation="longest_first"  # Choose a valid truncation strategy if needed
        )

# Display embeddings
    return embeddings

In [24]:
def create_vector_db(file_path: str, output_path: str = 'vector_db.json') -> None:
    """
    Creates a vector database from a text file and saves it to a JSON file.

    Args:
        file_path (str): The path to the text file containing the sentences.
        output_path (str, optional): The path where the JSON file will be saved. Defaults to 'vector_db.json'.

    Returns:
        None
    """


    # List to store lines from the text file
    lines = []

    # Read the text file and process each line
    with open(file_path, "r") as f:
        for line in f:
            # Strip leading and trailing whitespace, including newlines
            line = line.strip()
            # Skip empty lines
            if line:
                # Append the non-empty line to the list
                lines.append(line)

    # Generate embeddings for the lines
    embeddinglist = list(map(embed, lines))

    # Extract sparse activations from the embeddings
    tensor_list = [d['sparse_activations'] for d in embeddinglist if 'sparse_activations' in d]

    # Convert tensors to lists so they can be saved in JSON
    lists = [tensor.tolist() for tensor in tensor_list]

    # Create a dictionary to store arrays and sentences
    data = {"arrays": lists, "sentences": lines}

    # Save data to a JSON file
    with open(output_path, 'w') as f:
        json.dump(data, f)

    print(f"Vector database saved to {output_path}")

In [25]:
def load_vector_data(file_path: str) -> tuple[list[torch.tensor], list[str]]:

    """
    Loads vector data from a JSON file and converts it back to tensors.

    Args:
        file_path (str): The path to the JSON file containing the vector data.

    Returns:
        tuple: A tuple containing two elements:
            - loaded_tensors (list of torch.Tensor): A list of tensors converted from the stored lists.
            - loaded_sentences (list of str): A list of sentences corresponding to the vectors.
    """

    with open(file_path, 'r') as f:
        loaded_data = json.load(f)
    
    # Retrieve lists and sentences
    loaded_lists = loaded_data["arrays"]
    loaded_sentences = loaded_data["sentences"]
    
    # Convert lists back to tensors
    loaded_tensors = [torch.tensor(lst) for lst in loaded_lists]
    
    return loaded_tensors, loaded_sentences

In [26]:
def get_splade_embeddings(example_question: str) -> torch.tensor:

    """
    Encodes a given question into sparse embeddings using the Splade model.

    Args:
        example_question (str): The question to be encoded into embeddings.

    Returns:
        torch.Tensor: The sparse activations from the embeddings.
    """
    
    splade_model = sparsembed.model.Splade(
        model=AutoModelForMaskedLM.from_pretrained("naver/splade_v2_max"),
        tokenizer=AutoTokenizer.from_pretrained("naver/splade_v2_max")
    )

    # Encode text into embeddings using Splade
    with torch.no_grad():
        # Encode the input text into embeddings
        embeddings = splade_model.encode(
            texts=[example_question],  # Provide the input text as a list
            truncation="longest_first"  # Choose a valid truncation strategy if needed
        )

    return embeddings["sparse_activations"]

In [27]:
def find_most_similar_sentence(example_question: str, loaded_tensors: list[torch.tensor], loaded_sentences: list[str]) -> str:

    """
    Finds the most similar sentence to the given question from a list of pre-loaded sentences.

    Args:
        example_question (str): The question to compare against the loaded sentences.
        loaded_tensors (list of torch.Tensor): The list of tensors representing the embeddings of the loaded sentences.
        loaded_sentences (list of str): The list of pre-loaded sentences corresponding to the embeddings.

    Returns:
        str: The sentence from the loaded sentences that is most similar to the given question.
    """
    
    sparse_embedding_input = get_splade_embeddings(example_question)
    
    # Calculate cosine similarities
    similarities = [cosine_similarity(i, sparse_embedding_input.T)[0, 0] for i in loaded_tensors]
    
    # Find the index of the max similarity value
    max_value_index = similarities.index(max(similarities))
    
    # Return the most similar sentence
    return loaded_sentences[max_value_index]

## Gebruik van de gemaakte functies

Om de functionaliteit van de hierboven beschreven functies te demonstreren, zullen we een aantal taken uitvoeren.
Eerst maken we een vectordatabase json bestand. vervolgens gaan we de gemaakte functies gebruiken om de meest relevante zin op te halen.

In [28]:
# Creates the json file for the with the sentences and embeddings
create_vector_db(r"parsed.txt")

Vector database saved to vector_db.json


In [29]:
# Example usage of the functions created above
example_question = "How do I process biometric data and other personal data responsibly with AI for my company?"
file_path = 'vector_db.json'
loaded_tensors, loaded_sentences = load_vector_data(file_path) # deze duurt lang dus die miss ergens anders aanroepen en saven (dit is de database inladen)

result = find_most_similar_sentence(example_question, loaded_tensors, loaded_sentences)

# Print the result
print(result)

Article 3 | None | 33 | ‘biometric data’ means personal data resulting from specific technical processingrelating to the physical, physiological or behavioural characteristics of a natural person, such as facial images or dactyloscopic data;


### Top N similarities

In dit hoofdstuk breiden we onze analyse uit door de mogelijkheid toe te voegen om de top N meest vergelijkbare zinnen op te halen voor een gegeven vraag. We beginnen met het laden van de vooraf geladen vectorgegevens en zinnen uit een JSON-bestand, zoals eerder beschreven. Vervolgens implementeren we een functie die gebruikmaakt van cosine similarity om de zinnen te rangschikken en de top N resultaten te selecteren.

In [30]:
file_path = 'vector_db.json'
loaded_tensors, loaded_sentences = load_vector_data(file_path)

In [31]:
def find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3) -> pd.DataFrame:

    """
    Finds the top N most similar sentences to the given question from a list of pre-loaded sentences.

    Args:
        example_question (str): The question to compare against the loaded sentences.
        loaded_tensors (list of torch.Tensor): The list of tensors representing the embeddings of the loaded sentences.
        loaded_sentences (list of str): The list of pre-loaded sentences corresponding to the embeddings.
        top_n (int, optional): The number of top similar sentences to return. Defaults to 3.

    Returns:
        pd.DataFrame: A DataFrame containing the top N most similar sentences, their tensors, and their similarity scores.
    """
    sparse_embedding_input = get_splade_embeddings(example_question)
    
    # Calculate cosine similarities
    similarities = [cosine_similarity(i.unsqueeze(0), sparse_embedding_input.T)[0, 0] for i in loaded_tensors]
    
    # Create a DataFrame with sentences, tensors, and similarities
    df = pd.DataFrame({
        'sentences': loaded_sentences,
        'tensors': loaded_tensors,
        'similarity': similarities
    })

    # Sort by similarity and get the top N entries
    top_n_df = df.sort_values(by='similarity', ascending=False).head(top_n)
    
    return top_n_df

In [32]:
# Example usage of the function created above
example_question = "How do I process biometric data and other personal data responsibly with AI for my company?"
file_path = 'vector_db.json'
top_n_df = find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3)

# Display the result
top_n_df

,sentences,tensors,similarity
245,Article 3 | None | 33 | ‘biometric data’ means...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.40864322]
304,Article 7 | None | ba | the nature and amount ...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.3528345]
251,Article 3 | None | 35 | ‘biometric categorisat...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.33119974]


## Evaluatie van Robuustheid en Performantie

In dit hoofdstuk voeren we verschillende testcases uit om de robuustheid en performantie van ons systeem te evalueren. We richten ons specifiek op het gedrag van de functie die de meest vergelijkbare zinnen identificeert op basis van een gegeven vraag. Hierbij worden de volgende testcases behandeld:

Ongerelateerde Vraag: We testen hoe ons systeem omgaat met een vraag die weinig of geen overeenkomst heeft met de opgeslagen zinnen. Dit helpt ons te begrijpen of het systeem in staat is om onderscheid te maken tussen relevante en niet-relevante input.

Ongerelateerd Woord als Input: We onderzoeken hoe ons systeem reageert wanneer een enkelvoudig, ongerelateerd woord als input wordt gegeven in plaats van een volledige vraag. Dit testgeval helpt ons de capaciteit van het systeem te beoordelen om met onvolledige of onduidelijke input om te gaan.

Reeks Gerelateerde Woorden als Input: We stellen een reeks gerelateerde woorden als input voor. Dit scenario simuleert een situatie waarin de vraag complexer is en meerdere aspecten of contextuele informatie bevat. We evalueren of ons systeem in staat is om de juiste zinnen te identificeren die overeenkomen met de gegeven context.

Door verschillende types van input te testen, kunnen we beoordelen of ons systeem consistent blijft in het identificeren van relevante zinnen en het negeren van irrelevante informatie.

### Testcase 1: Ongerelateerde vragen

In [33]:
example_question = "Hi, how are you?"
top_n_df = find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3)

# Display the result
top_n_df

,sentences,tensors,similarity
267,Article 3 | None | 44c | ‘non-personal data’ m...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.06366091]
90,III. CONCLUSION | ANNEX | 47 | To address conc...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.041683216]
148,III. CONCLUSION | ANNEX | 69 | In order to fac...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.037700266]


In [34]:
# Tweede ongerelateerde vraag
example_question = "What different type of horses are there"
top_n_df = find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3)

# Display the result
top_n_df

,sentences,tensors,similarity
150,III. CONCLUSION | ANNEX | 70a | A variety of A...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.06646946]
764,article 52a | None | d | input and output moda...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.060234632]
578,Article 58b | None | iiii | providing advice o...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.05739121]


### Testcase2: een ongerelateerd woord als input

In [35]:
example_question = "Football"
top_n_df = find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3)

# Display the result
top_n_df

,sentences,tensors,similarity
226,Article 3 | None | 14 | ‘safety component of a...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.08789998]
214,Article 3 | None | 1a | ‘risk’ means the combi...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.08459283]
274,Article 3 | None | bk | ‘informed consent’ mea...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.07527994]


In [36]:
example_question = "apple pie"
top_n_df = find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3)

# Display the result
top_n_df

,sentences,tensors,similarity
227,Article 3 | None | 15 | ‘instructions for use’...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.09187493]
215,Article 3 | None | 2 | ‘provider’ means a natu...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.09030487]
213,Article 3 | None | 1 | ‘AI system‘ is a machin...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.08032041]


### Testcase 3: reeks gerelateerde woorden

In [37]:
example_question = "Biometric, Biometric, Biometric, Biometric"
top_n_df = find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3)

# Display the result
top_n_df

,sentences,tensors,similarity
245,Article 3 | None | 33 | ‘biometric data’ means...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.41524762]
251,Article 3 | None | 35 | ‘biometric categorisat...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.3791988]
247,Article 3 | None | 33c | ‘biometric verificati...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.37913036]


In [38]:
example_question = "risk, risk, risk, risk"
top_n_df = find_top_n_similar_sentences(example_question, loaded_tensors, loaded_sentences, top_n=3)

# Display the result
top_n_df

,sentences,tensors,similarity
214,Article 3 | None | 1a | ‘risk’ means the combi...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.53656644]
462,Article 52a | None | None | risk if it meets a...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.3569135]
296,Article 6 | None | None | independently from t...,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",[0.32530776]


## Conclusie en advies

### Conclusie
Na het uitvoeren van de testcases om de functionaliteit van het systeem te evalueren, kunnen we de volgende conclusies trekken:

#### Testcase 1: Ongerelateerde vragen
Voorbeeld: De input was gericht op niet-persoonlijke gegevens, zoals beschreven in juridische artikelen.
Resultaat: Het systeem toonde een lage similarity score voor alle zinnen, wat aangeeft dat het goed onderscheid maakt tussen gerelateerde en ongerelateerde vragen.

#### Testcase 2: Een ongerelateerd woord als input
Voorbeeld: Een enkelvoudig, niet-gerelateerd woord werd gebruikt als input, zoals "Football".
Resultaat: Het systeem produceerde zinnen met lage similarity scores.

#### Testcase 3: Reeks gerelateerde woorden
Voorbeeld: Een reeks gerelateerde woorden, zoals "biometric data", werd gebruikt als input.
Resultaat: Het systeem identificeerde zinnen met hogere similarity scores die specifiek gerelateerd waren aan het onderwerp "biometric data".

### Advies en verbetering

#### 1. Uitbreiden van de Testsample: 
Om een meer robuuste evaluatie uit te voeren, adviseren we het gebruik van een grotere testsample. Dit zal helpen bij het vaststellen van een geschikte drempelwaarde voor de similarity scores.

#### 2. Toevoegen van een threshold: 
Na het uitbreiden van de testsample is het een optie om een threshhold toe te voegen zodat RAG alleen tekst ophaalt die relevant genoeg is. Dit kan worden bereikt door een statistische analyse van de scores uit te voeren om een beter begrip te krijgen van wat als "relevant" moet worden beschouwd.

## Interesting sources

1. https://milvus.io/docs/embed-with-splade.md
2. https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/semantic-search/sparse/splade/splade-vector-generation.ipynb
3. https://zilliz.com/learn/discover-splade-revolutionize-sparse-data-processing
4. https://medium.com/@saschametzger/what-are-tokens-vectors-and-embeddings-how-do-you-create-them-e2a3e698e037
5. https://www.rungalileo.io/blog/mastering-rag-how-to-select-an-embedding-model
6. https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder
7. https://www.youtube.com/watch?v=wvk5uxMwMYs